In [ ]:
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'
import numpy as np
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Embedding, Dropout, PReLU
from keras.layers import Bidirectional, SpatialDropout1D, CuDNNGRU, Conv1D
from keras.layers import GlobalAvgPool1D, GlobalMaxPool1D, concatenate
from keras.regularizers import l2
from toxic.keras_utils import AttentionWithContext
from toxic.NN import NN

In [ ]:
class RnnVersion1(NN):
    """Input-> Embedding-> 2x(GRU-> Conv-> PReLU-> Attention)-> Output"""

    def __init__(self, model_name, embedding_file, max_seq_len=250):

        super().__init__(model_name, max_seq_len, file_dir='../../input',
                         word_embedding_file=embedding_file)
        self.load_labels()
        self.load_word_sequences()
        self.load_word_vectors()

    def get_model(self, n_recurrent=50, n_filters=50, dropout_rate=0.2, l2_penalty=0.0001):
        K.clear_session()

        def att_max_avg_pooling(x):
            x_att = AttentionWithContext()(x)
            x_avg = GlobalAvgPool1D()(x)
            x_max = GlobalMaxPool1D()(x)
            return concatenate([x_att, x_avg, x_max])
        
        inputs = Input(shape=(self.max_seq_len,))
        emb = Embedding(self.word_embedding_matrix.shape[0], 300, 
                        weights=[self.word_embedding_matrix], trainable=False)(inputs)
        
        # model 0
        x0 = SpatialDropout1D(dropout_rate)(emb)
        x0 = Bidirectional(
            CuDNNGRU(n_recurrent, return_sequences=True,
                     kernel_regularizer=l2(l2_penalty),
                     recurrent_regularizer=l2(l2_penalty)))(x0)
        x0 = Conv1D(n_filters, kernel_size=3)(x0)
        x0 = PReLU()(x0)
        x0 = att_max_avg_pooling(x0)

        # model 1
        x1 = SpatialDropout1D(dropout_rate)(emb)
        x1 = Bidirectional(
            CuDNNGRU(2*n_recurrent, return_sequences=True,
                     kernel_regularizer=l2(l2_penalty),
                     recurrent_regularizer=l2(l2_penalty)))(x1)
        x1 = Conv1D(2*n_filters, kernel_size=2)(x1)
        x1 = PReLU()(x1)
        x1 = att_max_avg_pooling(x1)

        # combine it all
        x = concatenate([x0, x1])
        outputs = Dense(6, activation='sigmoid')(x)
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(loss='binary_crossentropy', optimizer='nadam')
        return model

In [ ]:
param_lims = {
    'toxic': { 
        'n_filters':    54,
        'n_recurrent':  68,
        'dropout_rate': 0.052287,
        'l2_penalty':   2.1464e-05, 
    },
    'fasttext': {
        'n_filters':    95,
        'n_recurrent':  80,
        'dropout_rate': 0.1405,
        'l2_penalty':   0.001065, 
    },
    'glove': { 
        'n_filters':    90,
        'n_recurrent':  90,
        'dropout_rate': 0.1223,
        'l2_penalty':   0.003625, 
    },
    'lexvec': {
        'n_filters':    40,
        'n_recurrent':  62,
        'dropout_rate': 0.15334,
        'l2_penalty':   0.003158, 
    },
}
 

for emb in ['toxic', 'fasttext', 'glove', 'lexvec']:
    nn = RnnVersion1(
        model_name = f'RnnVersion1_{emb}',
        embedding_file = f'embeddings/{emb}_embeddings.txt')
    
    for seed in [0, 1, 2]:
        nn.predict_kfold(param_lims[emb], n_folds=10, seed=seed,
                         monitor_training=False, save_oof=True)